# Computational Methods in Economics

## Problem Set 7 - Stochastic Dynamic Programming

In [1]:
# Author: Alex Schmitt (schmitt@ifo.de)

import datetime
print('Last update: ' + str(datetime.datetime.today()))

Last update: 2018-01-29 11:12:07.067229


### Preliminaries

#### Import Modules

In [2]:
import numpy as np
import scipy.optimize

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn

import funapprox_cme as fa

## Question 1 (N)

Consider the following *stochastic* version of the NGM:

\begin{equation}
    \max_{\left\{c_t, k_{t+1}, h_{t} \right\}_{t = 0}^\infty} \mathbb{E} \sum^\infty_{t = 0} \beta^t \left[\frac{c_t^{1 - \nu}}{1 - \nu} - B \frac{h_t^{1+\eta}}{1+\eta}\right]
\end{equation}

s.t.

\begin{equation}
    c_t + k_{t + 1} =  z_t A k_t^\alpha h_{t}^{1-\alpha} + (1 - \delta) k_t
\end{equation}

with $z_t \in \{0.98, 1.02\}$. Compared to the standard model that we have seen so far, we assume that output is affected by a *productivity shock* $z_t$. In other words, $z_t$ is a discrete random variable (i.e. follows a stochastic process) whose "support" consists of two values, a low state $z_0 = 0.98$ and a high state $z_1 = 1.02$.

We assume that these states are "persistent" in the sense that if the economy is in one state in period $t$, it is more likely to be in the same state in period $t+1$, rather than to moving to a different state. We can formalize this with a *probability transition matrix*:

\begin{equation}
    P =  \left[\begin{array}{cc}
    p_{00} & p_{01} \\
    p_{10} & p_{11} \\
    \end{array}\right]
\end{equation}

The element $p_{ij}$ denotes the constant probability to move from state $i$ to state $j$ between two periods. Hence, it has to hold that $p_{00} + p_{01} = 1$ and $p_{10} + p_{11} = 1$.

(a) Think about solving this model using a sequential approach. Why is this much more computationally intensive than for the deterministic model we have seen before? In other words, how many variables would the algorithm have to find when solving the model for a finite number of periods $T$?

(b) We now solve the model using dynamic programming, for the parameter values given below:

In [3]:
## utility
beta = 0.8      # discount factor
nu = 2       # risk-aversion coefficient for consumption
eta = 1         # elasticity parameter for labor supply

## production
alpha = 0.25
delta = 0.1
## derived
A = (1 - beta * (1 - delta))/(alpha*beta) # normalization parameter for production function => steady state k = 1
B = (1 - alpha) * A * (A - delta)**(-nu)      # parameter for utility function
## initial capital stock
k0 = 1

params = {'beta': beta, 'nu': nu, 'eta': eta, 
          'alpha': alpha, 'delta': delta, 'A': A, 'B': B }

In [4]:
## stochastic states and transition matrix
z = [0.98 , 1.02]
P = np.array([[0.8, 0.2],
                [0.2, 0.8]])

Note that this model has two states, $k$ and $z$. However, in contrast to the two-state problem in question 4 in the previous problem set, $z$ is not a continuous variable, but can only take a *discrete* number of values (here 2). Hence, one approach to dynamic programming in this setting is to approximate the value function as a univariate function, but have two "instances" of $V$, one when the economy is in state 0 and one in state 1. 

In other words, we have approximations $\hat{V}^{(it)}_0(k) = \hat{V}(k; a^{(it)}_0)$ and $\hat{V}^{(it)}_1(k) = \hat{V}(k; a^{(it)}_1)$. This implies that the fitting step in every iteration of our value function iteration algorithm includes finding *two* vectors of basis coefficients.

Moreover, note that the continuation value on the right hand side of the Bellman equation is given by the expectation over the possible states. In other words, when we are in state 0, we remain in state 0 with probability $p_{00}$ and move to state 1 with probability $p_{01}$. Hence, the expected continuation value is 

\begin{equation}
    p_{00} \hat{V}(k'; a^{(it)}_0) + p_{01} \hat{V}(k'; a^{(it)}_1)
\end{equation}

and analogous if the economy is in state 1. Hence, the full Bellman equation to be used for the VFI algorithm when being in state $j \in \{0, 1\}$ reads 

\begin{equation}
    \hat{V}^{(it + 1)}_j(k) \leftarrow \max_{k', h} u(z_j f(k, h) + (1 - \delta) k - k') + \beta\left[p_{j0} \hat{V}^{(it)}_0(k') + p_{j1} \hat{V}^{(it)}_0(k')\right]
\end{equation}

In other words, we essentially have two Bellman equations, one for each state. 

Solve the problem and derive the approximate policy functions. Simulate the economy over $T = 80$ periods. To simulate a sequence of shocks, $z_t$, you can use the following code:

In [5]:
## number of periods
T = 80

## initialize array to keep track of time series
zt = np.zeros(T+1, dtype=np.int)

## initial shock 
zt[0] = 0

## loop over time and draw shocks
for t in range(T):
    draw = np.random.uniform()       
    
    if draw < P[zt[t], 0]:
        zt[t+1] = 0
    else:
        zt[t+1] = 1 

## Question 2 (N)

In this question, we look at a different application of dynamic programming: modeling job search. More specifically, we look at the famous *McCall job search model* (McCall 1970). The exposition follows the section on quantecon.org; for more information, see https://lectures.quantecon.org/py/mccall_model.html.

In any period $t$, an infinitely-lived worker can be in one of two states, employed or unemployed. In the former case, she receives income $y_t = w$ (a wage), in the latter case she receives $y_t = b$, a fixed unemployment benefit. Her lifeftime utility is given by 

\begin{equation}
{\mathbb E} \sum_{t=0}^\infty \beta^t u(y_t)
\end{equation}

where $\beta$ is a discount factor.

In any period $t$, when the worker is employed, she loses her job with a given probability $\alpha$. In this case, she has no decision to take. In the other state, if the worker starts period $t$ being unemployed, she receives a job offer with probability $\gamma$. Moreover, the wage corresponding to the offer is a random variable, drawn from a uniform distribution $p$, with the minimum and maximum level given by $\underline{w}$ and $\bar{w}$, respectively. The worker then decides whether or not to accept the offer. If she accepts, she starts the following period being employed, while if she rejects, she remains unemployed.

How can we write the Bellman equation for this model? As in question 1, we actually need two Bellman equations, one for each state. Let $V(w)$ denote the value of being employed with wage $w$, and let $U$ denote the value of being unemployed. Then, the Bellman equation when being employed reads:

\begin{equation}
    V(w) = u(w) + \beta\left[(1 - \alpha) V(w) + \alpha U \right] 
\end{equation}

Again, note that there is no maximization problem on the right-hand side, since the worker has no decision to take: if she doesn't lose her job, she receives the same wage in the following period. 

When being unemployed, the Bellman equation reads:

\begin{equation}
    U = u(b) + \beta \left[ (1 - \gamma) U + \gamma \int_{\underline{w}}^{\bar{w}} \max\left\{V(w'), U\right\}    p(w') dw'\right] 
\end{equation}

Consider the two possible cases when starting period $t$ unemployed:

- With probability $(1 - \gamma)$, the worker does not receive a job offer and remains unemployed at the start of period $t + 1$.

- With probability $\gamma$, she receives a job offer, with a wage drawn from distribution $p$. In order to find the expected continuation value, since the wage is a continuous variable, we have to integrate over the range of possible outcomes $[\underline{w}, \bar{w}]$. For each wage rate $w'$, the worker would then decide whether to accept the offer (in this case, her continuation  value is $V(w')$) or to reject it, in which case she remains unemployed and has continuation value $U$. The decision is based on what choice gives the higher continuation value.

Note that the value $U$ does not depend on any wage rate, and hence is actually a constant, rather than a function.

Computationally, this problem is in some sense easier than the DP problem in the NGM that we have seen before, mainly because of the optimization problem. In the NGM, we have maximized the RHS of the Bellman equation over continuous variables (capital and labor). Here, the worker faces a discrete choice: accept or reject the job offer. Hence, we do not need to use derivative-based optimization algorithms like BFGS, instead maximization here means essentially comparing the outcomes under the two options.

If we treat $w$ as a continuous variable, we would still approximate the value function using Chebyshev polynomials, and then use a numerical integration approach to evaluate the integral on the right-hand side of the second Bellman equation. However, since we have not talked about numerical integration in this class, we use a simpler approach. We assume that the wage can only take a finite number $N$ of values; hence, it is a discrete random variable. For a uniform distribution, the probability of a given value $w_i$ is $1/N$. We can then approximate the integral with a simple sum:

\begin{equation}
    U = u(b) + \beta \left[ (1 - \gamma) U + \gamma \frac{1}{N} \sum_{i = 1}^N \max\left\{V(w_i), U\right\}\right] 
\end{equation}

Use dynamic programming to solve the model for $N = 60$ possible values for the wage, equally spaced in the interval $[10, 20]$ and the parameter values given below. Plot the value functions against the possible wage rates. At what wage level does the worker accept an offer (in other words, what is the so-called *reservation wage*)?

In [6]:
def u(c, nu):
    return (c**(1 - nu) - 1) / (1 - nu)

alpha = 0.2      # Job separation rate
beta = 0.98        # Discount factor
gamma = 0.7       # Job offer rate
b = 6.0      # Unemployment compensation
nu = 2.0       # Utility parameter